In [2]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np

# from colorama import Fore, Style
from typing import List
from dotenv import load_dotenv
from managers.DataRepository import DataRepository
from ultralytics import YOLO

load_dotenv()
STORAGE_DIR = os.getenv("STORAGE_DIR")
CROPPED_VIDEOS_FOLDER = "cropped-videos"
# IMAGES_FOLDER = "images"
# LABELS_FOLDER = "labels"
SUPPORTED_VIDEO_FORMATS = os.getenv("SUPPORTED_VIDEO_FORMATS")
SUPPORTED_IMAGE_FORMATS = os.getenv("SUPPORTED_IMAGE_FORMATS")
VIDEO_IMAGE_PREVIEW_FOLDER = os.getenv("VIDEO_IMAGE_PREVIEW_FOLDER")


In [3]:
from moviepy import ImageSequenceClip

In [4]:
print(STORAGE_DIR), print(CROPPED_VIDEOS_FOLDER)
# computervision/labeled_frame_nrs_to_imgs.ipynb

/media/miked/Elements/Judge/FINISHED-DB-READY
cropped-videos


(None, None)

In [5]:
os.makedirs(os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER), exist_ok=True)

def get_video_path(repo, videoId):
    return os.path.join(STORAGE_DIR, repo.VideoNames.loc[videoId, "name"])

In [6]:
repo = DataRepository()

connection established, loading frame paths
relative paths of framelabels loaded


In [7]:
modeltrainround = 5
modelpath = f"/home/miked/code/judge/runs/detect/train{modeltrainround}" if modeltrainround > 0 else f"/home/miked/code/judge/runs/detect/train"
modelpath

'/home/miked/code/judge/runs/detect/train5'

In [8]:
# From https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode
# Load a model
model = YOLO(os.path.join(modelpath, "weights", "best.pt"))


In [9]:
[676]

[676]

In [14]:
DIM = 224
# videoIds = repo.get_dd3_videoIds()["id"].to_list()
videoIds = [676]

for videoId in videoIds:

    predictions_per_second = 2000
    videoPath = get_video_path(repo, videoId=videoId)
    videoOutputPath = os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER, f"{DIM}_{videoId}_opencv.mp4")
    videoOutputPathCorrect = os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER, f"{DIM}_{videoId}.mp4")
    
    if os.path.exists(videoOutputPathCorrect):
        continue

    cap = cv2.VideoCapture(videoPath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    wait_time = int(333 / fps - 7)
    frames_to_skip = fps // predictions_per_second
    frames_to_skip
    padding_x = width // 60
    padding_y = height // 50
    N = 3
    avgIOUlastNseconds = 0.0
    secondary_avgIOUlastNseconds = 0.0

    smootval = 0.85
    smootval_shrink = 0.945
    smooted_x1_min = None
    smooted_y1_min = None
    smooted_x2_max = None
    smooted_y2_max = None

    max_w = 0
    max_h = 0
    min_w = width
    min_h = height
    max_wh = max(width, height)

    i = 0
    times_with_no_jumper = 0
    cropping_started = False
    frames = []

    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')    # fourcc = 0x00000021
    # out = cv2.VideoWriter(videoOutputPath, fourcc, fps, (DIM, DIM))
    ret, frame = cap.read()
    while ret:
        i += 1
        result = model(frame, verbose=False)
        xyxy_boxes = result[0].boxes.xyxy
        
        if xyxy_boxes.shape[0] > 0:
            cropping_started = True
            x1_min = max(0, int(xyxy_boxes[:, 0].min().item()) - padding_x)
            y1_min = max(0, int(xyxy_boxes[:, 1].min().item()) - padding_y)
            x2_max = min(width, int(xyxy_boxes[:, 2].max().item()) + padding_x)
            y2_max = min(height, int(xyxy_boxes[:, 3].max().item()) + padding_y)
            times_with_no_jumper = 0
        elif times_with_no_jumper < 8 and 'x1_min' in locals():
            times_with_no_jumper += 1
        else:
            x1_min = 0
            y1_min = 0
            x2_max = width
            y2_max = height
            times_with_no_jumper += 1

        if smooted_x1_min is None or smooted_x1_min == 1:
            smooted_x1_min = x1_min
            smooted_y1_min = y1_min
            smooted_x2_max = x2_max
            smooted_y2_max = y2_max
            secondary_smooted_x1_min = x1_min
            secondary_smooted_y1_min = y1_min
            secondary_smooted_x2_max = x2_max
            secondary_smooted_y2_max = y2_max

        else:
            intersection_width = max(0, min(x2_max, smooted_x2_max) - max(x1_min, smooted_x1_min))
            intersection_height = max(0, min(y2_max, smooted_y2_max) - max(y1_min, smooted_y1_min))
            intersection_area = intersection_width * intersection_height
            area_new_box = (x2_max - x1_min) * (y2_max - y1_min)
            area_smooted_box = (smooted_x2_max - smooted_x1_min) * (smooted_y2_max - smooted_y1_min)
            union = area_smooted_box + area_new_box - intersection_area
            iou = intersection_area / union
            avgIOUlastNseconds = ((N * fps - 1) * avgIOUlastNseconds + iou) / N / fps 
            avgSqrd = avgIOUlastNseconds*avgIOUlastNseconds
            iou_threshold = avgSqrd * avgSqrd

            # secondary
            secondary_intersection_width = max(0, min(x2_max, secondary_smooted_x2_max) - max(x1_min, secondary_smooted_x1_min))
            secondary_intersection_height = max(0, min(y2_max, secondary_smooted_y2_max) - max(y1_min, secondary_smooted_y1_min))
            secondary_intersection_area = secondary_intersection_width * secondary_intersection_height
            area_new_box = (x2_max - x1_min) * (y2_max - y1_min)
            area_secondary_smooted_box = (secondary_smooted_x2_max - secondary_smooted_x1_min) * (secondary_smooted_y2_max - secondary_smooted_y1_min)
            secondary_union = area_secondary_smooted_box + area_new_box - secondary_intersection_area
            secondary_iou = secondary_intersection_area / secondary_union
            secondary_avgIOUlastNseconds = ((N / 2 * fps - 1) * secondary_avgIOUlastNseconds + iou) / N / 2 / fps 
            secondary_avgSqrd = secondary_avgIOUlastNseconds*secondary_avgIOUlastNseconds
            secondary_iou_threshold = secondary_avgSqrd * secondary_avgSqrd

            print(f"{iou > iou_threshold} : {iou:.3f} - 2nd = {secondary_iou:.3f} - T = {secondary_iou_threshold:.3f} --> follow? = {secondary_iou > secondary_iou_threshold}")
            # print(i, iou, iou_threshold)
            if iou > iou_threshold:
                smooted_x1_min = int(smootval * smooted_x1_min + (1-smootval) * x1_min) if x1_min < smooted_x1_min else int(smootval_shrink * smooted_x1_min + (1-smootval_shrink) * x1_min)
                smooted_y1_min = int(smootval * smooted_y1_min + (1-smootval) * y1_min) if y1_min < smooted_x1_min else int(smootval_shrink * smooted_y1_min + (1-smootval_shrink) * y1_min)
                smooted_x2_max = int(smootval * smooted_x2_max + (1-smootval) * x2_max) if x2_max > smooted_x2_max else int(smootval_shrink * smooted_x2_max + (1-smootval_shrink) * x2_max)
                smooted_y2_max = int(smootval * smooted_y2_max + (1-smootval) * y2_max) if y2_max > smooted_y2_max else int(smootval_shrink * smooted_y2_max + (1-smootval_shrink) * y2_max)
            secondary_smooted_x1_min = int(smootval * secondary_smooted_x1_min + (1-smootval) * x1_min) if x1_min < secondary_smooted_x1_min else int(smootval_shrink * smooted_x1_min + (1-smootval_shrink) * x1_min)
            secondary_smooted_y1_min = int(smootval * secondary_smooted_y1_min + (1-smootval) * y1_min) if y1_min < secondary_smooted_x1_min else int(smootval_shrink * smooted_y1_min + (1-smootval_shrink) * y1_min)
            secondary_smooted_x2_max = int(smootval * secondary_smooted_x2_max + (1-smootval) * x2_max) if x2_max > secondary_smooted_x2_max else int(smootval_shrink * smooted_x2_max + (1-smootval_shrink) * x2_max)
            secondary_smooted_y2_max = int(smootval * secondary_smooted_y2_max + (1-smootval) * y2_max) if y2_max > secondary_smooted_y2_max else int(smootval_shrink * smooted_y2_max + (1-smootval_shrink) * y2_max)

        w_jumpers = smooted_x2_max - smooted_x1_min
        h_jumpers = smooted_y2_max - smooted_y1_min
        max_w = max(max_w, w_jumpers)
        max_h = max(max_h, h_jumpers)
        min_w = min(min_w, w_jumpers)
        min_h = min(max_h, h_jumpers)


        max_wh_jumpers = max(w_jumpers, h_jumpers)
        offset_x = (max_wh_jumpers - w_jumpers) // 2
        offset_y = (max_wh_jumpers - h_jumpers) // 2

        leftover_pixels_x = smooted_x1_min - offset_x
        if leftover_pixels_x < 0:
            crop_x1 = 0
            offset_x = abs(leftover_pixels_x)
        else:
            crop_x1 = leftover_pixels_x
            offset_x = 0
        
        leftover_pixels_x_right = width - (smooted_x2_max + offset_x)
        crop_x2 = width if leftover_pixels_x_right < 0 else min(smooted_x2_max + offset_x, width)

        leftover_pixels_y = smooted_y1_min - offset_y
        if leftover_pixels_y < 0:
            crop_y1 = 0
            offset_y = abs(leftover_pixels_y)
        else:
            crop_y1 = leftover_pixels_y
            offset_y = 0
        
        leftover_pixels_y_right = width - (smooted_y2_max + offset_y)
        crop_y2 = width if leftover_pixels_y_right < 0 else min(smooted_y2_max + offset_y, height)

        cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]
        zeros = np.zeros((max_wh_jumpers, max_wh_jumpers, 3), dtype=np.uint8)
        zeros[offset_y:offset_y+(crop_y2-crop_y1), offset_x:offset_x+(crop_x2 - crop_x1)] = cropped_frame
        zeros = cv2.resize(zeros, (DIM, DIM))
        zeros = cv2.cvtColor(zeros, cv2.COLOR_BGR2RGB)

        frames.append(zeros)
        
        ret, frame = cap.read()

    print("min width", min_w, "max_w", max_w)   
    print("min_height", min_h, "max_h", max_h)
    print(i)

    # Release everything if job is finished
    cap.release()
    cv2.destroyAllWindows()

    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(videoOutputPathCorrect)

    # To browser runnable codec
    # os.system(f"ffmpeg -i {videoOutputPath} -vcodec libx264 {videoOutputPathCorrect}")

True : 0.995 - 2nd = 0.995 - T = 0.000 --> follow? = True
True : 0.991 - 2nd = 0.991 - T = 0.000 --> follow? = True
True : 0.987 - 2nd = 0.987 - T = 0.000 --> follow? = True
True : 0.990 - 2nd = 0.990 - T = 0.000 --> follow? = True
True : 0.992 - 2nd = 0.992 - T = 0.000 --> follow? = True
True : 0.982 - 2nd = 0.982 - T = 0.000 --> follow? = True
True : 0.990 - 2nd = 0.990 - T = 0.000 --> follow? = True
True : 0.988 - 2nd = 0.988 - T = 0.000 --> follow? = True
True : 0.990 - 2nd = 0.990 - T = 0.000 --> follow? = True
True : 0.975 - 2nd = 0.975 - T = 0.000 --> follow? = True
True : 0.987 - 2nd = 0.987 - T = 0.000 --> follow? = True
True : 0.981 - 2nd = 0.981 - T = 0.000 --> follow? = True
True : 0.983 - 2nd = 0.983 - T = 0.000 --> follow? = True
True : 0.978 - 2nd = 0.978 - T = 0.000 --> follow? = True
True : 0.984 - 2nd = 0.984 - T = 0.000 --> follow? = True
True : 0.984 - 2nd = 0.984 - T = 0.000 --> follow? = True
True : 0.990 - 2nd = 0.990 - T = 0.000 --> follow? = True
True : 0.982 -

MoviePy - Done !
MoviePy - video ready /media/miked/Elements/Judge/FINISHED-DB-READY/cropped-videos/224_676.mp4
